<a href="https://colab.research.google.com/github/ARAN1218/UsedcarValuePrediction_AI/blob/main/jupyter%20notebook/1.%E4%B8%AD%E5%8F%A4%E8%BB%8A%E4%BE%A1%E6%A0%BC%E4%BA%88%E6%B8%ACAI(%E4%BE%A1%E6%A0%BC_com)_%E3%82%B9%E3%82%AF%E3%83%AC%E3%82%A4%E3%83%94%E3%83%B3%E3%82%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Car_Bigdata

In [ ]:
from bs4 import BeautifulSoup
import requests
from time import sleep
import json
import pandas as pd
from tqdm.notebook import tqdm


pd.set_option('display.max_columns',None)


#pref[1~55] page[1~60]
#長丁場になりそう
page_start = 1
page_end = 60
car_start = 1
car_end = 20
pref_start = 1
pref_end = 55
pref_code = 55
url = "https://kakaku.com/kuruma/used/spec/Prefecture={}/Page={}/"

In [ ]:
pre_results = []

In [ ]:
#価格.comから中古車データを都道府県毎にスクレイピングするプログラム
#Used_Car_Data_Scraping
def UCDS(start,end,pref_code,pre_results=[]):
    #pre_resultsに値があった場合(再開する場合)、読み込んだページ数をprogressに記録
    d_list = pre_results
    if len(d_list) > 0:
        progress = list(pd.DataFrame(d_list)['ページ'].unique())
    else:
        progress = []
        
    print("Scraping Start!")

    #tqdmで進捗を監視
    for i in tqdm(range(start, end+1)):

        try:
            target_url = url.format(pref_code, i)
            print(target_url)
            r = requests.get(target_url)
            sleep(1)
            soupr = BeautifulSoup(r.content, 'html.parser')
            contents = soupr.find_all('div', class_="itemImgBox u-pRight15")
            #print(contents)
            name_index = 0
            for content in tqdm(contents):
                car_links = content.find_all("a")
                #print(car_links)
                names = soupr.find_all('a', onclick="if('function' == typeof window.onclickcatalyst) onclickcatalyst('spec_text');")
                #print(names[name_index].text)
                for car_link in car_links:
                    #既に読み込んだページは飛ばして効率化を図る
                    if (car_link.get('href')[18:-1]) in progress:
                        continue
                    
                    l = requests.get("https://kakaku.com"+car_link.get('href'))
                    sleep(1)
                    soupl = BeautifulSoup(l.content, 'html.parser')
                    info1 = soupl.find("table", class_="specList")
                    info2 = soupl.find("div", class_="carinfoArea")
                    #print(info1)

                    page = car_link.get('href')[18:-1]
                    #print(page)

                    name = names[name_index].text
                    name_index += 1
                    print(name)
                    value = soupl.find_all("span", class_="num")[1].text
                    #print(value)
                    total = soupl.find("span", class_="total").text
                    #print(total)
                    color = info1.find_all('td')[0].text
                    #print(color)
                    year = info1.find_all('td')[1].text
                    #print(year)
                    distance = info1.find_all('td')[2].text
                    #print(distance)
                    area = info1.find_all('td')[3].text
                    #print(area)
                    inspection = info1.find_all('td')[4].text
                    #print(inspection)
                    repair = info1.find_all('td')[5].text
                    #print(repair)
                    unused = info1.find_all('td')[6].text
                    #print(unused)
                    oneowner = info1.find_all('td')[7].text
                    #print(oneowner)
                    nosmoking = info1.find_all('td')[8].text
                    #print(nosmoking)
                    guarantee = info1.find_all('td')[9].text
                    #print(guarantee)

                    #print("\n")

                    mission = info2.find_all('dd')[0].text
                    #print(mission)
                    drive = info2.find_all('dd')[3].text
                    #print(drive)
                    displacement = info2.find_all('dd')[6].text
                    #print(displacement)
                    door = info2.find_all('dd')[9].text
                    #print(door)
                    capacity = info2.find_all('dd')[12].text
                    #print(capacity)
                    fuel = info2.find_all('dd')[15].text
                    #print(fuel)
                    handle = info2.find_all('dd')[1].text
                    #print(handle)
                    number = info2.find_all('dd')[4].text
                    #print(number)
                    record = info2.find_all('dd')[7].text
                    #print(record)
                    recycle = info2.find_all('dd')[10].text
                    #print(recycle)
                    maintenance = info2.find_all('dd')[13].text
                    #print(maintenance)
                    judgement = info2.find_all('dd')[16].text
                    #print(judgement)
                    length = info2.find_all('dd')[2].text
                    #print(length)
                    width = info2.find_all('dd')[5].text
                    #print(width)
                    height = info2.find_all('dd')[8].text
                    #print(height)
                    weight = info2.find_all('dd')[11].text
                    #print(weight)
                    fuelconsumption = info2.find_all('dd')[14].text
                    #print(fuelconsumption)
                    usingfuel = info2.find_all('dd')[17].text
                    #print(usingfuel)

                    #print("\n")
                    
                    options = soupl.find_all('li', class_='yes')
                    #print(options)
                    options_list = ""
                    for option in options:
                        #print(option.text)
                        options_list += str(option.text) + " "
                    #print(options_list)

                    d = {
                        "ページ":page
                        ,"車名":name

                        #info1
                        ,"価格":value
                        ,"総額":total
                        ,"色":color
                        ,"年式":year
                        ,"走行距離":distance
                        ,"地域":area
                        ,"車検":inspection
                        ,"修復歴":repair
                        ,"未使用車":unused
                        ,"ワンオーナー":oneowner
                        ,"禁煙車":nosmoking
                        ,"保証":guarantee

                        #info2
                        ,"ミッション":mission
                        ,"駆動方式":drive
                        ,"排気量":displacement
                        ,"ドア数":door
                        ,"乗車定員":capacity
                        ,"燃料":fuel
                        ,"ハンドル":handle
                        ,"車台番号":number
                        ,"記録簿":record
                        ,"リサイクル料":recycle
                        ,"整備":maintenance
                        ,"ID車両(グー鑑定)":judgement
                        ,"全長":length
                        ,"全幅":width
                        ,"全高":height
                        ,"車両重量":weight
                        ,"燃費":fuelconsumption
                        ,"使用燃料":usingfuel
                        
                        #info3
                        ,"オプション":options_list
                    }

                    d_list.append(d)

        #リストにある存在しないページにアクセスした場合、そのページの読み込みをスキップする。
        except IndexError:
            continue

        #何らかの理由で中断された場合、その時点までに読み込んだデータを保存する。
        except:
            break

    print("Scraping Completed!")
    return d_list


In [ ]:
results = UCDS(page_start, page_end, pref_code, pre_results)
pre_results = results
display(pd.DataFrame(pre_results))

In [ ]:
d_df = pd.DataFrame(results)
for name in list(d_df.columns):
    print(d_df[name].unique())
    print(d_df[name].nunique())

In [ ]:
d_df[d_df["オプション"].str.contains('福祉車両')]

In [ ]:
d_df.to_pickle('Car_{}.pickle'.format(pref_code))

In [ ]:
display(pd.read_pickle(f'Car_{pref_code}.pickle'))

In [ ]:
#スクレイピングメモ(オプションまで終わった都道府県)
#1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30
#31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55
#Scraping Complited!

In [ ]:
#urlエンコード　テスト
#テスラ = %83e%83X%83%89

import urllib.parse
tesla = "テスラ"
tesla_quote = urllib.parse.quote(tesla, encoding='shift-jis')
print(tesla_quote)
#これ応用したら任意の車のデータが自動でスクレイピング出来るようになるのでは